## Kaggle Titanic CNN

## Data Dictionary
#### VariableDefinitionKey: 
- survival Survival 0 = No, 1 = Yes pclass Ticket class 1 = 1st, 2 = 2nd, 3 = 3rd 
- sex Sex Age Age in years sibsp # of siblings / spouses aboard the Titanic
- parch # of parents / children aboard the Titanic ticket Ticket number fare Passenger fare cabin Cabin number 
- embarked Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton

#### Variable Notes

- pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

- age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

- sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

- parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
import seaborn as sns

In [2]:
# Importing the dataset
dataset = pd.read_csv('C:/Users/gavam/Desktop/ml_BookLab/Titanic/titanic_data/train.csv')
dataset1 = pd.read_csv('C:/Users/gavam/Desktop/ml_BookLab/Titanic/titanic_data/test.csv')
dataset2 = pd.read_csv('C:/Users/gavam/Desktop/ml_BookLab/Titanic/titanic_data/gender_submission.csv')

In [3]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
y_test = pd.read_csv('C:/Users/gavam/Desktop/ml_BookLab/Titanic/titanic_data/gender_submission.csv').values
y_train = dataset[['Survived']].values

In [5]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values="NaN",strategy="mean",axis=0)
imputer = imputer.fit(dataset[['Age']])
dataset[['Age']] = imputer.transform(dataset[['Age']])

imputer = Imputer(missing_values="NaN",strategy="mean",axis=0)
imputer = imputer.fit(dataset1[['Age']])
dataset1[['Age']] = imputer.transform(dataset1[['Age']])


dataset[['SibSp','Parch','Fare']] = dataset[['SibSp','Parch','Fare']].apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))
dataset1[['SibSp','Parch','Fare']] = dataset1[['SibSp','Parch','Fare']].apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))

In [6]:
'''DATA PREPERATION-MANIPULATION'''
dataset['Cabin'] = dataset['Cabin'].replace(regex={r'[^NaN]': 1})
dataset1['Cabin'] = dataset1['Cabin'].replace(regex={r'[^NaN]': 1})

dataset['Fam_num'] = dataset['SibSp'] + dataset['Parch']
dataset1['Fam_num'] = dataset1['SibSp'] + dataset1['Parch']

In [7]:
dataset['Fam_num'] = np.where(dataset['Fam_num'].between(1,3), 2, dataset['Fam_num'])
dataset['Fam_num'] = np.where(dataset['Fam_num'].between(4,5), 3, dataset['Fam_num'])
dataset['Fam_num'] = np.where(dataset['Fam_num'].between(5,20), 4, dataset['Fam_num'])

dataset1['Fam_num'] = np.where(dataset1['Fam_num'].between(2,3), 2, dataset1['Fam_num'])
dataset1['Fam_num'] = np.where(dataset1['Fam_num'].between(4,5), 3, dataset1['Fam_num'])
dataset1['Fam_num'] = np.where(dataset1['Fam_num'].between(5,20), 4, dataset1['Fam_num'])

In [8]:
maxfare = max(dataset['Fare'])
minfare = min(dataset['Fare'])
meanfare = np.mean(dataset['Fare'])

maxfare = max(dataset['Fare'])
minfare = min(dataset['Fare'])
meanfare = np.mean(dataset['Fare'])

In [9]:
'''AGE IN CATEGORIES'''
dataset['Age'] = np.where(dataset['Age'].between(0,15), 1, dataset['Age'])
dataset['Age'] = np.where(dataset['Age'].between(15,30), 2, dataset['Age'])
dataset['Age'] = np.where(dataset['Age'].between(30,50), 3, dataset['Age'])
dataset['Age'] = np.where(dataset['Age'].between(50,100), 4, dataset['Age'])


dataset1['Age'] = np.where(dataset1['Age'].between(0,15), 1, dataset1['Age'])
dataset1['Age'] = np.where(dataset1['Age'].between(15,30), 2, dataset1['Age'])
dataset1['Age'] = np.where(dataset1['Age'].between(30,50), 3, dataset1['Age'])
dataset1['Age'] = np.where(dataset1['Age'].between(50,100), 4, dataset1['Age'])

In [10]:
"""dataset = pd.concat([dataset,pd.get_dummies(dataset['Pclass'], prefix='Pclass', drop_first=True)], axis=1)
dataset1 = pd.concat([dataset1,pd.get_dummies(dataset1['Pclass'], prefix='Pclass', drop_first=True)], axis=1)
dataset.drop(['Pclass'],axis=1, inplace=True)
dataset1.drop(['Pclass'],axis=1, inplace=True)"""
dataset['Pclass'] = np.where(dataset['Pclass'].between(0,2), 0, dataset['Pclass'])
dataset['Pclass'] = np.where(dataset['Pclass'].between(3,4), 1, dataset['Pclass'])
dataset1['Pclass'] = np.where(dataset1['Pclass'].between(0,2), 0, dataset1['Pclass'])
dataset1['Pclass'] = np.where(dataset1['Pclass'].between(3,4), 1, dataset1['Pclass'])

dataset = pd.concat([dataset,pd.get_dummies(dataset['Age'], prefix='Age', drop_first=True)], axis=1)
dataset1 = pd.concat([dataset1,pd.get_dummies(dataset1['Age'], prefix='Age', drop_first=True)], axis=1)
#dataset.drop(['Age'],axis=1, inplace=True)
#dataset1.drop(['Age'],axis=1, inplace=True)

In [11]:
dataset['Fare'] = np.where(dataset['Fare'].between(0,meanfare/3), 1, dataset['Fare'])
dataset['Fare'] = np.where(dataset['Fare'].between(meanfare/3,maxfare), 0, dataset['Fare'])

dataset1['Fare'] = np.where(dataset1['Fare'].between(0,meanfare/3), 1, dataset1['Fare'])
dataset1['Fare'] = np.where(dataset1['Fare'].between(meanfare/3,maxfare), 0, dataset1['Fare'])

In [12]:
#values = {'Sex': 1, 'Cabin': 0, 'Embarked': 'S'}
X_train = dataset.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))
X_test = dataset1.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))
#X_train = dataset.fillna(value=values)
#X_test = dataset1.fillna(value=values)

In [13]:
X_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Fam_num', 'Age_2.0',
       'Age_3.0', 'Age_4.0'],
      dtype='object')

In [14]:
X_train = X_train.replace(['female','male'],[0,1])
X_test = X_test.replace(['female','male'],[0,1])

X_train = pd.concat([X_train,pd.get_dummies(X_train['Embarked'], prefix='Embarked', drop_first=True)], axis=1)
X_test = pd.concat([X_test,pd.get_dummies(X_test['Embarked'], prefix='Embarked', drop_first=True)], axis=1)

X_train = pd.concat([X_train,pd.get_dummies(X_train['Fam_num'], prefix='Fam_num', drop_first=True)], axis=1)
X_test = pd.concat([X_test,pd.get_dummies(X_test['Fam_num'], prefix='Fam_num', drop_first=True)], axis=1)

In [15]:
X_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Fam_num', 'Age_2.0',
       'Age_3.0', 'Age_4.0', 'Embarked_Q', 'Embarked_S', 'Fam_num_2',
       'Fam_num_3', 'Fam_num_4'],
      dtype='object')

In [16]:
X_train= X_train[['Sex', 'Cabin', 'Pclass', 
       'Age_2.0', 'Age_3.0','Age_4.0','Fam_num_2', 'Fam_num_3', 'Fam_num_4','Fare']]
X_test= X_test[['Sex', 'Cabin', 'Pclass', 
       'Age_2.0', 'Age_3.0','Age_4.0','Fam_num_2', 'Fam_num_3', 'Fam_num_4','Fare']]

In [17]:
#np.count_nonzero(np.isnan(X_train))
# performing standard scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [18]:
np.count_nonzero(np.isnan(X_train))

0

### Lets see some NN

In [19]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [20]:
from keras import Sequential
from keras.layers import Dense

C:\Users\gavam\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=10))
#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [23]:
eval_model=classifier.fit(X_train, y_train, epochs=300)

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
891/891 [==============================] - 0s 290us/step - loss: 0.6918 - acc: 0.6128
Epoch 2/300
891/891 [==============================] - 0s 35us/step - loss: 0.6889 - acc: 0.6162
Epoch 3/300
891/891 [==============================] - 0s 28us/step - loss: 0.6856 - acc: 0.6162
Epoch 4/300
891/891 [==============================] - 0s 28us/step - loss: 0.6799 - acc: 0.6442
Epoch 5/300
891/891 [==============================] - 0s 29us/step - loss: 0.6691 - acc: 0.7834
Epoch 6/300
891/891 [==============================] - 0s 30us/step - loss: 0.6519 - acc: 0.7845
Epoch 7/300
891/891 [==============================] - 0s 27us/step - loss: 0.6317 - acc: 0.7666
Epoch 8/300
891/891 [==============================] - 0s 30us/step - loss: 0.6108 - acc: 0.7744
Epoch 9/300
891/891 [==============================] - 0s 29us/step - loss: 0.5936 - acc: 0.7856
Epoch 10/300
891/891 [==============================] - 0s 26us/step - loss: 0

891/891 [==============================] - 0s 25us/step - loss: 0.4287 - acc: 0.8316
Epoch 82/300
891/891 [==============================] - 0s 26us/step - loss: 0.4284 - acc: 0.8316
Epoch 83/300
891/891 [==============================] - 0s 29us/step - loss: 0.4284 - acc: 0.8316
Epoch 84/300
891/891 [==============================] - 0s 24us/step - loss: 0.4278 - acc: 0.8316
Epoch 85/300
891/891 [==============================] - 0s 27us/step - loss: 0.4275 - acc: 0.8316
Epoch 86/300
891/891 [==============================] - 0s 25us/step - loss: 0.4272 - acc: 0.8316
Epoch 87/300
891/891 [==============================] - 0s 24us/step - loss: 0.4268 - acc: 0.8316
Epoch 88/300
891/891 [==============================] - 0s 28us/step - loss: 0.4262 - acc: 0.8316
Epoch 89/300
891/891 [==============================] - 0s 27us/step - loss: 0.4257 - acc: 0.8316
Epoch 90/300
891/891 [==============================] - 0s 29us/step - loss: 0.4254 - acc: 0.8316
Epoch 91/300
891/891 [===========

891/891 [==============================] - 0s 28us/step - loss: 0.4089 - acc: 0.8395
Epoch 165/300
891/891 [==============================] - 0s 27us/step - loss: 0.4090 - acc: 0.8395
Epoch 166/300
891/891 [==============================] - 0s 29us/step - loss: 0.4089 - acc: 0.8395
Epoch 167/300
891/891 [==============================] - 0s 24us/step - loss: 0.4083 - acc: 0.8395
Epoch 168/300
891/891 [==============================] - 0s 34us/step - loss: 0.4086 - acc: 0.8395
Epoch 169/300
891/891 [==============================] - 0s 30us/step - loss: 0.4080 - acc: 0.8395
Epoch 170/300
891/891 [==============================] - 0s 27us/step - loss: 0.4083 - acc: 0.8395
Epoch 171/300
891/891 [==============================] - 0s 30us/step - loss: 0.4084 - acc: 0.8395
Epoch 172/300
891/891 [==============================] - 0s 27us/step - loss: 0.4080 - acc: 0.8395
Epoch 173/300
891/891 [==============================] - 0s 26us/step - loss: 0.4077 - acc: 0.8395
Epoch 174/300
891/891 [=

891/891 [==============================] - 0s 29us/step - loss: 0.4045 - acc: 0.8395
Epoch 247/300
891/891 [==============================] - 0s 25us/step - loss: 0.4054 - acc: 0.8395
Epoch 248/300
891/891 [==============================] - 0s 29us/step - loss: 0.4047 - acc: 0.8395
Epoch 249/300
891/891 [==============================] - 0s 27us/step - loss: 0.4047 - acc: 0.8395
Epoch 250/300
891/891 [==============================] - 0s 25us/step - loss: 0.4048 - acc: 0.8395
Epoch 251/300
891/891 [==============================] - 0s 25us/step - loss: 0.4041 - acc: 0.8395
Epoch 252/300
891/891 [==============================] - 0s 25us/step - loss: 0.4044 - acc: 0.8395
Epoch 253/300
891/891 [==============================] - 0s 28us/step - loss: 0.4043 - acc: 0.8395
Epoch 254/300
891/891 [==============================] - 0s 27us/step - loss: 0.4045 - acc: 0.8395
Epoch 255/300
891/891 [==============================] - 0s 29us/step - loss: 0.4042 - acc: 0.8395
Epoch 256/300
891/891 [=

In [24]:
y_pred=classifier.predict(X_test)
y_pred = np.squeeze(y_pred)

In [25]:
y_pred = y_pred>0.5

In [26]:
y_pred = y_pred.astype(int)    
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [27]:
y_test2 = [y[1] for y in y_test]

In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test2, y_pred)
print(cm)

[[256  10]
 [ 30 122]]


In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test2, y_pred)

0.9043062200956937

In [133]:
dataset2 = pd.read_csv('C:/Users/gavam/Desktop/ml_BookLab/Titanic/titanic_data/gender_submission.csv')
y_test = dataset2.values
predictions = pd.DataFrame(y_test2, columns = ['Survived'])
X = pd.concat([dataset2['PassengerId'],predictions['Survived']], axis=1)

In [134]:
X.set_index('PassengerId').to_csv('predictions_NN.csv')